In [1]:
import ray


In [3]:
class SimpleClass:
    def __init__(self, value):
        self.value = value

    def increase(self):
        self.value += 1
        print(self.value)
ray.init()      
SimpleClass = ray.remote(SimpleClass)
obj = SimpleClass.remote(2)
obj.increase.remote()
getattr(obj, "increase").remote()
ray.shutdown()

ObjectRef(63964fa4841d4a2e59229e3d057d393869f45ab50100000001000000)

(SimpleClass pid=98259) 3


In [3]:
import sys
sys.path.append('../')
from fl_strategies.fed_avg import FedAvg
import yaml
from fedrec.utilities.logger import NoOpLogger
from fedrec.python_executors.aggregator import AggregatorConfig

from fedrec.multiprocessing.process_manager import RayProcessManager
from executor import JobExecutor
with open("../configs/dlrm_fl.yml", 'r') as cfg:
    config = yaml.load(cfg, Loader=yaml.FullLoader)

ag_config = {
        # Seed for RNG used in shuffling the training data.
    "data_seed" : 100,
    # Seed for RNG used in initializing the model.
    "init_seed" : 100,
    # Seed for RNG used in computing the model's training loss.
    # Only relevant with internal randomness in the model, e.g. with dropout.
    "model_seed" : 100
}
aggregator_cfg = AggregatorConfig(**ag_config)

In [6]:
try:
    ray.init()
except Exception as e:
    ray.shutdown()
    ray.init()
    
JobExecutor = ray.remote(JobExecutor)
je_Aggregator = JobExecutor.remote(FedAvg,
                                   config["model"],
                                   aggregator_cfg,
                                   NoOpLogger())


(pid=102367) 2021-11-14 21:25:04,646	ERROR serialization.py:270 -- No module named 'fl_strategies'
(pid=102367) Traceback (most recent call last):
(pid=102367)   File "/home/varun/miniconda/envs/recoedge/lib/python3.8/site-packages/ray/serialization.py", line 268, in deserialize_objects
(pid=102367)     obj = self._deserialize_object(data, metadata, object_ref)
(pid=102367)   File "/home/varun/miniconda/envs/recoedge/lib/python3.8/site-packages/ray/serialization.py", line 191, in _deserialize_object
(pid=102367)     return self._deserialize_msgpack_data(data, metadata_fields)
(pid=102367)   File "/home/varun/miniconda/envs/recoedge/lib/python3.8/site-packages/ray/serialization.py", line 169, in _deserialize_msgpack_data
(pid=102367)     python_objects = self._deserialize_pickle5_data(pickle5_data)
(pid=102367)   File "/home/varun/miniconda/envs/recoedge/lib/python3.8/site-packages/ray/serialization.py", line 159, in _deserialize_pickle5_data
(pid=102367)     obj = pickle.loads(in_band)

In [4]:
rpm = RayProcessManager()
rpm.distribute(JobExecutor, FedAvg.__name__, 1 ,
                            FedAvg, config["model"], aggregator_cfg, NoOpLogger())

rpm.start(FedAvg.__name__, "run")

KeyError: 'FedAvg'

(pid=101966) 2021-11-14 21:18:11,629	ERROR serialization.py:270 -- No module named 'fl_strategies'
(pid=101966) Traceback (most recent call last):
(pid=101966)   File "/home/varun/miniconda/envs/recoedge/lib/python3.8/site-packages/ray/serialization.py", line 268, in deserialize_objects
(pid=101966)     obj = self._deserialize_object(data, metadata, object_ref)
(pid=101966)   File "/home/varun/miniconda/envs/recoedge/lib/python3.8/site-packages/ray/serialization.py", line 191, in _deserialize_object
(pid=101966)     return self._deserialize_msgpack_data(data, metadata_fields)
(pid=101966)   File "/home/varun/miniconda/envs/recoedge/lib/python3.8/site-packages/ray/serialization.py", line 169, in _deserialize_msgpack_data
(pid=101966)     python_objects = self._deserialize_pickle5_data(pickle5_data)
(pid=101966)   File "/home/varun/miniconda/envs/recoedge/lib/python3.8/site-packages/ray/serialization.py", line 159, in _deserialize_pickle5_data
(pid=101966)     obj = pickle.loads(in_band)

In [5]:
ray.shutdown()